<a href="https://colab.research.google.com/github/duraichellam001-tech/epa-co2-architecture/blob/master/notebooks/01_epa_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 10tstcar.csv to 10tstcar.csv
Saving 11tstcar.csv to 11tstcar.csv
Saving 12tstcar.csv to 12tstcar.csv
Saving 13tstcar.csv to 13tstcar.csv
Saving 14tstcar.csv to 14tstcar.csv
Saving 15tstcar.csv to 15tstcar.csv
Saving 16tstcar.csv to 16tstcar.csv
Saving 17tstcar.xlsx to 17tstcar.xlsx
Saving 18tstcar.xlsx to 18tstcar.xlsx
Saving 19tstcar.xlsx to 19tstcar.xlsx
Saving 20tstcar.xlsx to 20tstcar.xlsx
Saving 21tstcar.xlsx to 21tstcar.xlsx
Saving 22tstcar.xlsx to 22tstcar.xlsx
Saving 23tstcar.xlsx to 23tstcar.xlsx
Saving 24tstcar.xlsx to 24tstcar.xlsx
Saving 25tstcar.xlsx to 25tstcar.xlsx


In [4]:
import glob
import os

DATA_DIR = "/content"

files = (
    glob.glob(os.path.join(DATA_DIR, "*.csv")) +
    glob.glob(os.path.join(DATA_DIR, "*.xlsx"))
)

print(f"Found {len(files)} files")


Found 16 files


In [23]:
REQUIRED_COLUMNS = [
    "Model Year",
    "Test Veh Displacement (L)",
    "Equivalent Test Weight (lbs.)",
    "Test Category",
    "Tested Transmission Type",
    "Drive System Code",
    "Test Fuel Type Description",
    "CO2 (g/mi)"
]


In [24]:
import pandas as pd

dfs = []

for file in files:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
    else:
        df = pd.read_excel(file)

    # keep only columns that exist in this file
    cols_present = [c for c in REQUIRED_COLUMNS if c in df.columns]
    df = df[cols_present]

    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
print("Combined shape:", df_all.shape)


Combined shape: (70083, 8)


In [25]:
df_all["Test Fuel Type Description"].value_counts()


,count
Test Fuel Type Description,
Tier 2 Cert Gasoline,57134
Federal Cert Diesel 7-15 PPM Sulfur,3462
Electricity,2916
E85 (85% Ethanol 15% EPA Unleaded Gasoline),2306
Cold CO Regular (Tier 2),1361
Cold CO Premium (Tier 2),1306
CARB Phase II Gasoline,1207
Hydrogen 5,94
Cold CO Regular (CERT),87


In [26]:
# make everything lowercase for safe string matching
fuel_col = df_all["Test Fuel Type Description"].str.lower()

# boolean mask for gasoline fuels
is_gasoline = (
    fuel_col.str.contains("gasoline") |
    fuel_col.str.contains("cold co") |
    fuel_col.str.contains("indolene")
)


is_excluded = (
    fuel_col.str.contains("electric") |
    fuel_col.str.contains("diesel") |
    fuel_col.str.contains("e85") |
    fuel_col.str.contains("cng") |
    fuel_col.str.contains("lpg") |
    fuel_col.str.contains("hydrogen")
)

# final gasoline-only mask
df_gasoline = df_all[is_gasoline & (~is_excluded)].copy()

print("After gasoline filter shape:", df_gasoline.shape)


After gasoline filter shape: (61227, 8)


In [27]:
df_gasoline["Test Fuel Type Description"].value_counts()


,count
Test Fuel Type Description,
Tier 2 Cert Gasoline,57134
Cold CO Regular (Tier 2),1361
Cold CO Premium (Tier 2),1306
CARB Phase II Gasoline,1207
Cold CO Regular (CERT),87
Tier 3 E10 Premium Gasoline (9 RVP @Low Alt.),37
Cold CO Premium (CERT),30
CARB LEV3 E10 Regular Gasoline,16
EPA Unleaded Gasoline,14


In [29]:
df_gasoline["Test Category"].value_counts()


,count
Test Category,
FTP,28204
HWY,24939
US06,4364
SC03,3083
CD,637


In [30]:
df_ch = df_gasoline[df_gasoline["Test Category"].isin(["FTP", "HWY"])].copy()

print(df_ch["Test Category"].value_counts())
print("Shape after keeping FTP + HWY:", df_ch.shape)


Test Category
FTP    28204
HWY    24939
Name: count, dtype: int64
Shape after keeping FTP + HWY: (53143, 8)


In [31]:
VARIANT_KEY = [
    "Model Year",
    "Test Veh Displacement (L)",
    "Equivalent Test Weight (lbs.)",
    "Tested Transmission Type",
    "Drive System Code",
]


df_ch.groupby(VARIANT_KEY)["Test Category"].nunique().value_counts()



,count
Test Category,
2,9653
1,13


In [33]:
VARIANT_KEY = [
    "Model Year",
    "Test Veh Displacement (L)",
    "Equivalent Test Weight (lbs.)",
    "Tested Transmission Type",
    "Drive System Code"
]

df_pivot = (
    df_ch
    .pivot_table(
        index=VARIANT_KEY,
        columns="Test Category",
        values="CO2 (g/mi)",
        aggfunc="mean"
    )
    .reset_index()
)

print(df_pivot.shape)
df_pivot.head(10)


(9500, 7)


Test Category,Model Year,Test Veh Displacement (L),Equivalent Test Weight (lbs.),Tested Transmission Type,Drive System Code,FTP,HWY
0,2010,0.999,2125,Automated Manual,R,201.260000,154.190000
1,2010,1.300,3000,Continuously Variable,F,162.374183,144.001838
2,2010,1.300,3125,Continuously Variable,F,162.953045,136.703344
3,2010,1.300,3375,Automatic,R,411.000000,263.000000
4,2010,1.300,3375,Manual,R,421.000000,276.333333
5,2010,1.497,2625,Automatic,F,239.244685,185.630962
6,2010,1.497,2625,Manual,F,234.013326,176.248686
7,2010,1.500,2750,Manual,F,246.180000,187.560000
8,2010,1.500,2875,Automatic,F,243.010000,180.400000
9,2010,1.500,2875,Manual,F,251.340000,194.000000


In [34]:
df_pivot = df_pivot.dropna(subset=["FTP", "HWY"]).copy()
print("After dropping incomplete variants:", df_pivot.shape)


After dropping incomplete variants: (9483, 7)


In [35]:
df_vehicle = df_pivot.copy()

df_vehicle["CO2_combined"] = (
    0.55 * df_vehicle["FTP"] +
    0.45 * df_vehicle["HWY"]
)

df_vehicle.shape, df_vehicle.head()


((9483, 8),
 Test Category  Model Year  Test Veh Displacement (L)  \
 0                    2010                      0.999   
 1                    2010                      1.300   
 2                    2010                      1.300   
 3                    2010                      1.300   
 4                    2010                      1.300   
 
 Test Category  Equivalent Test Weight (lbs.) Tested Transmission Type  \
 0                                       2125         Automated Manual   
 1                                       3000    Continuously Variable   
 2                                       3125    Continuously Variable   
 3                                       3375                Automatic   
 4                                       3375                   Manual   
 
 Test Category Drive System Code         FTP         HWY  CO2_combined  
 0                             R  201.260000  154.190000    180.078500  
 1                             F  162.374183  144.00

In [37]:
def map_transmission(x):
    x = str(x).lower()
    if "manual" in x and "automated" not in x:
        return "MT"
    if "variable" in x or "cvt" in x:
        return "CVT"
    return "AT"

df_vehicle["transmission_bucket"] = df_vehicle["Tested Transmission Type"].apply(map_transmission)

df_vehicle["transmission_bucket"].value_counts()



,count
transmission_bucket,
AT,6844
MT,1487
CVT,1152


In [38]:
pd.crosstab(
    df_vehicle["Tested Transmission Type"],
    df_vehicle["transmission_bucket"]
)


transmission_bucket,AT,CVT,MT
Tested Transmission Type,,,
Automated Manual,323,0,0
Automated Manual- Selectable (e.g. Automated Manual with paddles),618,0,0
Automatic,3253,0,0
Continuously Variable,0,797,0
Manual,0,0,1487
Other,17,0,0
Selectable Continuously Variable (e.g. CVT with paddles),0,355,0
Semi-Automatic,2633,0,0


In [41]:
def map_drive(x):
    x = str(x).upper()
    if x == "F":
        return "FWD"
    if x == "R":
        return "RWD"
    return "AWD"   # includes AWD, 4WD, others

df_vehicle["drive_bucket"] = df_vehicle["Drive System Code"].apply(map_drive)

df_vehicle["drive_bucket"].value_counts()


,count
drive_bucket,
FWD,3989
RWD,3659
AWD,1835


In [42]:
pd.crosstab(
    df_vehicle["Drive System Code"],
    df_vehicle["drive_bucket"]
)


drive_bucket,AWD,FWD,RWD
Drive System Code,,,
4,534,0,0
A,1193,0,0
F,0,3989,0
P,108,0,0
R,0,0,3659


In [43]:
final_columns = [
    "Model Year",
    "Test Veh Displacement (L)",
    "Equivalent Test Weight (lbs.)",
    "transmission_bucket",
    "drive_bucket",
    "CO2_combined"
]

df_model = df_vehicle[final_columns].copy()

print(df_model.shape)
df_model.head()


(9483, 6)


Test Category,Model Year,Test Veh Displacement (L),Equivalent Test Weight (lbs.),transmission_bucket,drive_bucket,CO2_combined
0,2010,0.999,2125,AT,RWD,180.078500
1,2010,1.300,3000,CVT,FWD,154.106628
2,2010,1.300,3125,CVT,FWD,151.140680
3,2010,1.300,3375,AT,RWD,344.400000
4,2010,1.300,3375,MT,RWD,355.900000


In [44]:
df_model.columns


Index(['Model Year', 'Test Veh Displacement (L)',
       'Equivalent Test Weight (lbs.)', 'transmission_bucket', 'drive_bucket',
       'CO2_combined'],
      dtype='object', name='Test Category')

In [45]:
dataset_metadata = {
    "dataset_name": "epa_co2_architecture_v1",
    "version": "v1.0",
    "rows": df_model.shape[0],
    "features": list(df_model.columns),
    "fuel_scope": "Gasoline ICE only",
    "years": "2010-2025",
    "aggregation": "Vehicle-level, FTP/HWY weighted 55/45",
    "excluded": [
        "EVs", "Diesel", "E85", "CNG", "LPG", "Hydrogen",
        "US06", "SC03", "CD",
        "MPG", "Road-load coefficients"
    ]
}

dataset_metadata


{'dataset_name': 'epa_co2_architecture_v1',
 'version': 'v1.0',
 'rows': 9483,
 'features': ['Model Year',
  'Test Veh Displacement (L)',
  'Equivalent Test Weight (lbs.)',
  'transmission_bucket',
  'drive_bucket',
  'CO2_combined'],
 'fuel_scope': 'Gasoline ICE only',
 'years': '2010-2025',
 'aggregation': 'Vehicle-level, FTP/HWY weighted 55/45',
 'excluded': ['EVs',
  'Diesel',
  'E85',
  'CNG',
  'LPG',
  'Hydrogen',
  'US06',
  'SC03',
  'CD',
  'MPG',
  'Road-load coefficients']}

In [46]:
DATASET_PATH = "/content/epa_co2_architecture_v1.parquet"
df_model.to_parquet(DATASET_PATH, index=False)

DATASET_PATH


'/content/epa_co2_architecture_v1.parquet'

In [47]:
import json

META_PATH = "/content/epa_co2_architecture_v1_metadata.json"
with open(META_PATH, "w") as f:
    json.dump(dataset_metadata, f, indent=2)

META_PATH


'/content/epa_co2_architecture_v1_metadata.json'